# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [ ]:
%pip install tira>=0.0.139 ir-datasets python-terrier==0.10.0

# llm stuff for later:

%pip install transformers
%pip install torch

# bei t5
%pip install sentencepiece

Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [ ]:
import pyterrier as pt

pt_dataset = pt.get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

print(len(pt_dataset.get_topics('text')))

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [ ]:
import os

indexer = pt.IterDictIndexer(
    index_path=os.getcwd() + os.sep + "index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)
index = indexer.index(pt_dataset.get_corpus_iter())

### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [4]:

bm25 = pt.BatchRetrieve(index, wmodel="BM25")



### Step 5





In [ ]:


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def extract_info(text):
    prompt = "Identify and remove all sentences that contain advertisements, promotional language, headlines, navigartions or URLs, and only keep the informative content from the following text: "
    input_text = prompt + text
    # Tokenisiere den kombinierten Text
    inputs = tokenizer(input_text, return_tensors="pt")
    #print("input", text)
    
    # Generiere eine Antwort basierend auf dem Text
    outputs = model.generate(inputs['input_ids'], max_length=1024, num_return_sequences=1)

    # Dekodiere die Antwort in Text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Rückgabe des extrahierten Textes
    #print("output", generated_text)
    return generated_text

    
def _docCleaning(docs):
    # Gehe jedes Dokument durch
    for index, text in enumerate(docs["text"]):
        text = extract_info(text)
    return docs

    
bm25_dc = pt.BatchRetrieve(index, wmodel="BM25") % 200 >> pt.text.get_text(pt_dataset, "text") >> _docCleaning >> pt.BatchRetrieve(index, wmodel="BM25")
# bm25_dc = pt.BatchRetrieve(index, wmodel="BM25") >> pt.apply.doc_score(_docCleaning) >> pt.BatchRetrieve(index, wmodel="BM25")

### Step 6: Experiment and Run


In [ ]:
pt.Experiment([bm25, bm25_dc],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    eval_metrics = ["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"],
    names= ["BM25", "BM25+LLMCleaning"]
)

In [ ]:
run = bm25_dc(pt_dataset.get_topics('text'))
run.head(10)

In [ ]:
from tira.third_party_integrations import persist_and_normalize_run

save_path = os.getcwd() + os.sep + "saved_runs" + os.sep
os.makedirs(save_path, exist_ok=True)

persist_and_normalize_run(
    run,
    # Give your approach a short but descriptive name tag.
    system_name='th25-LLMCleaning-t5FlanSmall', 
    default_output=save_path,
    upload_to_tira=pt_dataset,
)